# About These Notebooks

Unlike the [online Tellurium documentation](http://tellurium.readthedocs.io/en/latest), which is intended to work in **both** Jupyter and the Tellurium notebook viewer, these example notebooks are specialized for the Tellurium's notebook viewer. This allows you to use Tellurium's special notebook cells. Whereas regular Jupyter notebooks only contain Python code, Tellurium allows you to incorporate the community standards [SBML](http://sbml.org/Main_Page) and [COMBINE archives](http://co.mbine.org/documents/archive) directly in your notebook.

## Simple Example

This example shows how to set up a simple model in Tellurium and solve it as an ODE. Tellurium uses a human-readable representation of SBML models called [Antimony](http://antimony.sourceforge.net/). The following notebook cell is actually Antimony code - the Tellurium notebook viewer allows you to configure notebook cells as either Antimony, Python, or COMBINE archives. To create Antimony cells, simply select "Convert to Model Cell" from the dropdown in the upper right of a code cell.

When you run the cell below, it will create an instance of the [RoadRunner](http://sys-bio.github.io/roadrunner/python_docs/index.html) simulator. You can use this instance to simulate the model by running the `simulate` function with the start time, end time, and number of points.

In [1]:
model simple()
  S1 -> S2; k1*S1
  k1 = 0.1
  S1 = 10
end

In [2]:
simple.simulate(0, 50, 100)
simple.plot()

## More Complex Example

Tellurium can also handle stochastic models. This example shows how to select Tellurium’s stochastic solver. The underlying simulation engine used by Tellurium implements a Gibson direct method for simulating this model.

In this example, we run multiple simulations of a stochastic model. Then, we average the simulation trajectories and plot the average trajectory along with the ensemble.

In [3]:
model stochastic()
    J1: S1 -> S2;  k1*S1;
    J2: S2 -> S3; k2*S2 - k3*S3
    J3: S3 -> S4; k4*S3;

    k1 = 0.1; k2 = 0.5; k3 = 0.5; k4 = 0.5;
    S1 = 100;
end

In [4]:
import tellurium as te, numpy as np

# use a stochastic solver
stochastic.integrator = 'gillespie'
stochastic.integrator.seed = 1234
# selections specifies the output variables in a simulation
selections = ['time'] + stochastic.getBoundarySpeciesIds() + stochastic.getFloatingSpeciesIds()
stochastic.integrator.variable_step_size = False

# run repeated simulation
Ncol = len(stochastic.selections)
Nsim = 30
points = 101
s_sum = np.zeros(shape=[points, Ncol])
for k in range(Nsim):
    stochastic.resetToOrigin()
    s = stochastic.simulate(0, 50, points, selections=selections)
    s_sum += s
    # use show=False to add traces to the current plot
    # instead of starting a new one, equivalent to MATLAB hold on
    stochastic.plot(s, alpha=0.5, show=False)

# add mean curve, legend, show everything and set labels, titles, ...
fig = te.plot(s[:,0], s_sum[:,1:]/Nsim, names=[x + ' (mean)' for x in selections[1:]], title="Stochastic simulation", xtitle="time", ytitle="concentration")